In [47]:
import scipy
from scipy.stats.stats import pearsonr
from datetime import datetime
import statsmodels.api as sm
import numpy as np
import pandas as pd

In [48]:
#import statsmodels.api as sm
#import numpy as np
#import pandas as pd

MAX_PVAL=0.05
class StatisticalEDA:
    def __init__(self,edadf,file_object,logger_object):
        self.edadf=edadf
        self.file_object = file_object
        self.logger_object = logger_object
    """
    Method Description: This method will be used to get the OLS summary of the dataset. The variables having lower p-value will be kept and others will be dropped
    Input parameter names:self,dataframe,target variable
    Input Parameter Description: dataframe: the input data loaded from the source
    target: the dependent variable
    output:OLS Summary
    On Exception: Write the exception in the log file.
    Raise an exception with the appropriate error message
    Written by : Suvikram
    Version: 1.0.0
    Revision: None
    
    """
    def olsSummaryMethod(self,edadf,target):

        self.logger_object.log(self.file_object,'Entered into the OLS Summary Method')
        #Considering only the names of the features in the list of column names
        dfColumns=[icol for icol in edadf.columns if icol!=target]
        y_targ=edadf[target]
        #remvoing the target column and non-numeric types from the dataframe
        edadf=edadf[dfColumns]
        edadf=edadf.select_dtypes('number')

        numVars=len(dfColumns)

        for icol in range(0, numVars):
            try:
                #Running the OLS summary report
                regressor_OLS = sm.OLS(y_targ, edadf).fit()
                self.logger_object.log(self.file_object,'Able to run the OLS method for the dataset')
            except Exception as e:
                self.logger_object.log(self.file_object,'Exception faced during running the OLS method'+str(e))
                self.logger_object.log(self.file_object,'OLS method exited with the error')
                raise Exception
            '''
            maxVar = float(max(regressor_OLS.pvalues))
            if maxVar > MAX_PVAL:
                #Removing the column which has exceeded the threshold p value
                for jcol in range(0, numVars - icol):
                    if (regressor_OLS.pvalues[jcol].astype(float) == maxVar):
                        self.edadf = np.delete(self.edadf, jcol, 1)
                        self.logger_object.log(self.file_object,'Removed the column '+dfColumns[jcol]+'successfully')
                        dfColumns = np.delete(dfColumns, jcol) '''
            self.logger_object.log(self.file_object, 'OLS summary and pvalues are returned successfully')
        return regressor_OLS
    def get_corelation(self,dataframe,target,threshold = 0.60):
        
        """
    Method Description: This method will be used to remove multicolinearity from the dataframe
    Input parameter names:dataframe,target, Threshold
    Input Parameter Description: dataframe: the input data loaded from the source
    target: the dependent variable
    threshold: threshold to remove colinearity
    output:colinearity free dataframe
    On Exception: Write the exception in the log file.
    Raise an exception with the appropriate error message
    Written by : Shakti Kumar
    Version: 1.0.0
    Revision: None
    
    """
        self.logger_object.log(self.file_object,'Entered into the OLS Summary Method')
        try:
            dk = dataframe
            ols = self.olsSummaryMethod(dk,target)
            columnlist = list(dk.select_dtypes(include = 'number').columns)
            columnlist.remove(target)
            todrop = set()
            for i in range(0,len(columnlist)):
                for j in range(i):
                    pearson_coeff, pvalue = pearsonr(dk[columnlist[i]],dk[columnlist[j]])
                    if (pearson_coeff > threshold) or (pearson_coeff < (-1) * threshold) :
                        if ols.pvalues[columnlist[i]] > ols.pvalues[columnlist[j]]:
                            todrop.add(columnlist[i])
                        else:
                            todrop.add(columnlist[j])
            return dk.drop(todrop,axis = 1)
        
            
        except Exception as e:
                self.logger_object.log(self.file_object,'Exception faced during running the correlation method'+str(e))
                self.logger_object.log(self.file_object,'Correlation method exited with the error')
                raise Exception
        
        

In [49]:
class App_Logger:
    def __init__(self):
        pass

    def log(self, file_object, log_message):#This method will be used for logging all the information to the file.
        self.now = datetime.now()
        self.date = self.now.date()
        self.current_time = self.now.strftime("%H:%M:%S")
        file_object.write(str(self.date) + "/" + str(self.current_time) + "\t\t" + log_message +"\n")

In [50]:
f = open('dd.txt','a+')

In [51]:
loggerobj = App_Logger()

In [52]:
df = pd.read_csv('train.csv')

In [53]:
df.dropna(inplace = True)

In [54]:
df.shape

(48660, 14)

In [55]:
obj = StatisticalEDA(df,f,loggerobj)

In [56]:
data = obj.get_corelation(df,'is_promoted',0.60)

In [25]:
from datetime import datetime

In [57]:
data

employee_id         department     region         education gender  \
0            65438  Sales & Marketing   region_7  Master's & above      f   
1            65141         Operations  region_22        Bachelor's      m   
2             7513  Sales & Marketing  region_19        Bachelor's      m   
3             2542  Sales & Marketing  region_23        Bachelor's      m   
4            48945         Technology  region_26        Bachelor's      m   
...            ...                ...        ...               ...    ...   
54802         6915  Sales & Marketing  region_14        Bachelor's      m   
54803         3030         Technology  region_14        Bachelor's      m   
54804        74592         Operations  region_27  Master's & above      f   
54805        13918          Analytics   region_1        Bachelor's      m   
54807        51526                 HR  region_22        Bachelor's      m   

      recruitment_channel  no_of_trainings  age  previous_year_rating  \
0                sourcing                1   35                   5.0   
1                   other                1   30                   5.0   
2                sourcing                1   34                   3.0   
3                   other                2   39                   1.0   
4                   other                1   45                   3.0   
...                   ...              ...  ...                   ...   
54802               other                2   31                   1.0   
54803            sourcing                1   48                   3.0   
54804               other                1   37                   2.0   
54805               other                1   27                   5.0   
54807               other                1   27                   1.0   

       KPIs_met >80%  awards_won?  avg_training_score  is_promoted  
0                  1            0                  49            0  
1                  0            0                  60            0  
2                  0            0                  50            0  
3                  0            0                  50            0  
4                  0            0                  73            0  
...              ...          ...                 ...          ...  
54802              0            0                  49            0  
54803              0            0                  78            0  
54804              0            0                  56            0  
54805              1            0                  79            0  
54807              0            0                  49            0  

[48660 rows x 13 columns]